In [ ]:
import json
import pandas as pd
import numpy as np
import os
from openai import OpenAI
from tqdm import tqdm
import ast

In [ ]:
import sys
sys.path.append('/Users/mn/Desktop/BridgeAthletics/Proj2/Proj2DataPreprocessing')
from Functions.DataSetManipulation import json_read


In [ ]:
##################################################################

In [ ]:
with open("/Users/mn/Desktop/BridgeAthletics/config.json", "r") as config_file:
    config = json.load(config_file)
    api_key = config["OPENAI_API_KEY"]

client = OpenAI(api_key=api_key)

In [ ]:
def run_chatgpt(prompt, client, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    return response.choices[0].message.content

In [ ]:
############################ CHOOSE APPROPRIATE DATA SET DIRECTORY #############################

In [ ]:
dir='/Users/mn/Desktop/BridgeAthletics/Proj2/Proj2DataPreprocessing/'
sub_dir = ['Data1','Data2'] #if you create new dataset (dataset_i), sub_dir needs to be array of [dataset_i,dataset_i-1]

In [ ]:
############################ REMOVING USELESS BLOCK NAMES ####################################

In [ ]:
df = pd.DataFrame()
for sd in range(0, len(sub_dir)):
    temp_df = pd.read_csv(dir + sub_dir[sd] + '/IO_Workout_Sample.csv')
    df = pd.concat([df, temp_df], ignore_index=True)

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.isna().sum()

In [ ]:
df['name'] = df['name'].apply(ast.literal_eval)

In [ ]:
df.loc[:, 'info'] = df['info'].apply(lambda x: x.replace("nan", "[]") if isinstance(x, str) else x)
df.loc[:, 'info'] = df['info'].apply(lambda x: x.replace("None", "'None'") if isinstance(x, str) else x)

In [ ]:
df['info'] = df['info'].apply(lambda x: x.replace("nan", "[]").replace("None", "[]") if isinstance(x, str) else x)

def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError) as e:
        print(f"Error evaluating: {val}\nException: {e}")
        return val

df['info'] = df['info'].apply(lambda x: safe_literal_eval(x) if isinstance(x, str) else x)

In [ ]:
c=0
t=[]
for index, i in df['name'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        t.append(index)
df.drop(t, inplace=True)

In [ ]:
df.head()

In [ ]:
out_data = []
from copy import deepcopy
def custom_deepcopy(df):
    new_df = df.copy(deep=True)
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list) or isinstance(x, dict)).any():
            new_df[col] = df[col].apply(lambda x: deepcopy(x))
    return new_df

in_df = custom_deepcopy(df)

In [ ]:
for index, row in in_df.iterrows():
    ID=row['workoutID']
    indexes_to_remove=[]
    names=[]
    items=[]
    for ind,name in enumerate(row['name']):
        if 'warm' in name.lower() or 'prep' in name.lower():
            names.append(name)
            items.append(row['info'][ind])
            indexes_to_remove.append(ind)
    out_data.append({'workoutID': ID, 'name': names, 'info':items})
    for i in sorted(indexes_to_remove, reverse=True):
        in_df.at[index, 'name'].pop(i)
        in_df.at[index, 'info'].pop(i)

out_df = pd.DataFrame(out_data)     
out_df.head()

In [ ]:
in_df.head()

In [ ]:
print(len(in_df),len(out_df))

In [ ]:
c=0
for index, i in in_df['name'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
print('should be max len -',c)

c=0
for index, i in out_df['name'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
print(c)
c=0
for index, row in in_df.iterrows():
    if len(row['info'])!=len(row['name']):
        c=c+1
print(c)
c=0
for index, row in out_df.iterrows():
    if len(row['info'])!=len(row['name']):
        c=c+1
print(c)

In [ ]:
in_df.rename(columns={'name': 'main_ex_names', 'info': 'main_info'}, inplace=True)
out_df.rename(columns={'name': 'warmup_and_prep_ex_names', 'info': 'warmup_and_prep_info'}, inplace=True)

In [ ]:
print("in_df 'workoutID' is unique:", in_df['workoutID'].is_unique)
    

In [ ]:
IO_df = pd.merge(in_df, out_df, on='workoutID', how='left')

In [ ]:
c=0
for index, i in IO_df['main_ex_names'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
print('should be max len -',c)

c=0
for index, i in IO_df['warmup_and_prep_ex_names'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
print(c)
c=0
for index, row in IO_df.iterrows():
    if len(row['warmup_and_prep_info'])!=len(row['warmup_and_prep_ex_names']):
        c=c+1
print(c)
c=0
for index, row in IO_df.iterrows():
    if len(row['main_info'])!=len(row['main_ex_names']):
        c=c+1
print(c)

In [ ]:
print(len(IO_df),len(in_df),len(out_df))

In [ ]:
IO_df['main_ex_names'].head(n=10)

In [ ]:
for index, row in IO_df.iterrows():
    indexes_to_remove=[]
    for ind,name in enumerate(row['main_ex_names']):
        if 'recovery' in name.lower() or 'regeneration' in name.lower() or 'core' in name.lower() or 'auxiliary' in name.lower():
            indexes_to_remove.append(ind)
    for i in sorted(indexes_to_remove, reverse=True):
        IO_df.at[index, 'main_ex_names'].pop(i)
        IO_df.at[index, 'main_info'].pop(i)

In [ ]:
c=0
for index, i in IO_df['main_ex_names'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
print('should be max len -',c)

c=0
for index, i in IO_df['warmup_and_prep_ex_names'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
print(c)
c=0
for index, row in IO_df.iterrows():
    if len(row['warmup_and_prep_info'])!=len(row['warmup_and_prep_ex_names']):
        c=c+1
print(c)
c=0
for index, row in IO_df.iterrows():
    if len(row['main_info'])!=len(row['main_ex_names']):
        c=c+1
print(c)

In [ ]:
rows_to_delete = IO_df[IO_df['main_ex_names'].apply(len) < 1].index
print(rows_to_delete)
IO_df.drop(rows_to_delete, inplace=True)
IO_df.reset_index(drop=True, inplace=True)
IO_df[IO_df['main_ex_names'].apply(len) < 1].index

In [ ]:
for index, ex_names in IO_df['warmup_and_prep_ex_names'].items():
    indices_to_remove = [i for i, s in enumerate(ex_names) if 'personalized warmup' in s.lower()]
    IO_df.at[index, 'warmup_and_prep_ex_names'] = [s for i, s in enumerate(ex_names) if i not in indices_to_remove]
    IO_df.at[index, 'warmup_and_prep_info'] = [info for i, info in enumerate(IO_df.at[index, 'warmup_and_prep_info']) if i not in indices_to_remove]

In [ ]:
print(len(IO_df))

In [ ]:
c=0
for index, i in IO_df['main_ex_names'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
print('should be max len -',c)

c=0
rows_to_delete=[]
for index, i in IO_df['warmup_and_prep_ex_names'].items():
    if all('warm' not in s.lower() for s in i) and all('prep' not in s.lower() for s in i):
        c=c+1
        rows_to_delete.append(index)
    
print(f"Number of rows to delete: {len(rows_to_delete)}")
IO_df = IO_df.drop(rows_to_delete).reset_index(drop=True)
print(c-len(rows_to_delete))
c=0
for index, row in IO_df.iterrows():
    if len(row['warmup_and_prep_info'])!=len(row['warmup_and_prep_ex_names']):
        c=c+1
print(c)
c=0
for index, row in IO_df.iterrows():
    if len(row['main_info'])!=len(row['main_ex_names']):
        c=c+1
print(c)

In [ ]:
import pandas as pd

IO_df['main_info_str'] = IO_df['main_info'].apply(lambda x: json.dumps(x))
IO_df['warmup_and_prep_info_str'] = IO_df['warmup_and_prep_info'].apply(lambda x: json.dumps(x))

grouped = IO_df[IO_df.duplicated(subset=['main_info_str', 'warmup_and_prep_info_str'], keep=False)]

grouped_by = grouped.groupby(['main_info_str', 'warmup_and_prep_info_str'])
s=0
pd.set_option('display.max_colwidth', 150)
IO_df.drop(columns=['main_info_str', 'warmup_and_prep_info_str'], inplace=True)
for name, group in grouped_by:
    s=s+len(group)
    
print(s)
print(len(grouped_by))
pd.reset_option('display.max_colwidth')

In [ ]:
IO_df['main_info_str'] = IO_df['main_info'].apply(lambda x: json.dumps(x))
IO_df['warmup_and_prep_info_str'] = IO_df['warmup_and_prep_info'].apply(lambda x: json.dumps(x))

IO_df_unique = IO_df.drop_duplicates(subset=['main_info_str', 'warmup_and_prep_info_str']).copy()
IO_df_unique.drop(columns=['main_info_str', 'warmup_and_prep_info_str'], inplace=True)
IO_df_unique.reset_index(drop=True, inplace=True)

In [ ]:
print(len(IO_df_unique))
print(s+(len(IO_df_unique)-len(grouped_by))==len(IO_df))

In [ ]:
IO_df_unique.head()

In [ ]:
lengths = IO_df_unique['warmup_and_prep_info'].apply(lambda x: len(x))
lengths2= IO_df_unique['warmup_and_prep_ex_names'].apply(lambda x: len(x))
print(lengths[lengths-lengths2!=0])
max_length = 0

m=lengths2.apply(lambda x: x==max_length)
print(len(m[m]))

In [ ]:
#run for max_length=0 and other if needed
IO_df_unique.drop(m[m].index, inplace=True)
IO_df_unique.reset_index(drop=True, inplace=True)

In [ ]:
def extract_main_set_exercises(main_info):
    exercises = set()
    for block in main_info:
        for exo in block:
            if exo['exerciseType'][0].lower() == 'main set':
                exercises.add(exo['exercise'])
    return list(exercises)

IO_df_unique['main_exercises'] = IO_df_unique['main_info'].apply(extract_main_set_exercises)

In [ ]:
IO_df_unique.drop(columns=['main_info','main_ex_names'], inplace=True)
IO_df_unique = IO_df_unique[['workoutID', 'main_exercises', 'warmup_and_prep_ex_names', 'warmup_and_prep_info']]
IO_df_unique.head()

In [ ]:
IO_df_unique.to_pickle(dir + sub_dir[-1] +'/IO_Workout_TempSave.pkl')

In [ ]:
IO_df_unique=pd.read_pickle(dir + sub_dir[-1] +'/IO_Workout_TempSave.pkl')

In [ ]:
#NEXT: combine earmup names with info and remove null params

In [ ]:
def clean_exercise_info(blocks):
    final_block=[]
    for block in blocks:
        cleaned_info = []
        for entry in block:
            entry.pop('exerciseType', None)
            entry = {k: v for k, v in entry.items() if not (isinstance(v, list) and all(x == 0 for x in v))}
            cleaned_info.append(entry)
        final_block.append(cleaned_info)
    return final_block
    
IO_df_unique['warmup_and_prep_info'] = IO_df_unique['warmup_and_prep_info'].apply(clean_exercise_info)

In [ ]:
IO_df_unique['warmup_and_prep_ex_names'][1]

In [ ]:
def combine_exercises(names, info):
    return {name: exercises for name, exercises in zip(names, info)}
IO_df_unique['combined_warmup_and_prep'] = IO_df_unique.apply(lambda row: combine_exercises(row['warmup_and_prep_ex_names'], row['warmup_and_prep_info']), axis=1)
IO_df_unique.drop(columns=['warmup_and_prep_ex_names', 'warmup_and_prep_info'], inplace=True)



In [ ]:
IO_df_unique.head()

In [ ]:
IO_df_unique.to_pickle(dir + sub_dir[-1] +'/IO_Workout_Final.pkl')

In [ ]:
IO_df_unique=pd.read_pickle(dir + sub_dir[-1] +'/IO_Workout_Final.pkl')
IO_df_unique.drop(columns=['workoutID'], inplace=True)
IO_df_unique.rename(columns={'main_exercises':'input', 'combined_warmup_and_prep': 'output'}, inplace=True)

In [ ]:
rows_to_delete=set()
for index, i in IO_df_unique['input'].items():
    if len(i)==0:
        rows_to_delete.add(index)
        
for index, i in IO_df_unique['output'].items():
    if len(i)==0:
        rows_to_delete.add(index)

for index, i in IO_df_unique['output'].items():
    for k in i:
        if len(i[k])==0:
            rows_to_delete.add(index)
            
        
rows_to_delete=list(rows_to_delete)
        
print(f"Number of rows to delete: {len(rows_to_delete)}")
IO_df_unique = IO_df_unique.drop(rows_to_delete).reset_index(drop=True)

In [ ]:
print(len(IO_df_unique))

In [ ]:
new_json_file = dir + sub_dir[-1] +'/IO_Workout_Final.json'

In [ ]:
json_data = IO_df_unique.to_dict(orient='records')
with open(new_json_file, "w") as file:
    json.dump(json_data, file, indent=1)

In [ ]:
###READING DATASET

In [ ]:
with open(new_json_file, "r") as file:
    loaded_json_data = json.load(file)

In [ ]:
loaded_json_data[1]['input']
loaded_json_data[1]['output']['Warmup']
loaded_json_data[0]['input'][0]

In [ ]:
c=0
d=0
for i in range(len(loaded_json_data)-1,-1,-1):
    c=c+1
    if len(loaded_json_data[i]['input'])==0:
        d=d+1
        del(loaded_json_data[i])
    else:
        if len(loaded_json_data[i]['output'])==0:
            d=d+1
            del loaded_json_data[i]
        else:
            keys_to_remove = [k for k in loaded_json_data[i]['output'] if len(loaded_json_data[i]['output'][k]) == 0]
            if len(keys_to_remove)>0:
                d=d+1
                for k in keys_to_remove:
                    del loaded_json_data[i]['output'][k]
            
print('done')
print(c,d)

In [ ]:
c=0
d=0
for i in range(len(loaded_json_data)-1,-1,-1):
    c=c+1
    if (len(loaded_json_data[i]['input']) == 0 or len(loaded_json_data[i]['output'])==0):
        d=d+1
print('done')
print(c,d)

In [ ]:
print(len(loaded_json_data))